## Testing and timing of `moarchiving.BiobjectiveNondominatedSortedList`

In [1]:
import doctest
import moarchiving
# reload(moarchiving)
NA = moarchiving.BiobjectiveNondominatedSortedList
doctest.testmod(moarchiving)
# NA.make_expensive_asserts = True

TestResults(failed=0, attempted=44)

In [2]:
a = moarchiving.BiobjectiveNondominatedSortedList(
    [[-0.749, -1.188], [-0.557, 1.1076],
     [0.2454, 0.4724], [-1.146, -0.110]], [10, 10])
a._asserts()
a

[[-1.146, -0.11], [-0.749, -1.188]]

In [3]:
a.dominators([1, 3]) == a

True

In [4]:
a.add([-1, -3])  # return index where the element was added

1

In [5]:
a

[[-1.146, -0.11], [-1, -3]]

In [6]:
a.add([-1.5, 44])

In [7]:
a

[[-1.146, -0.11], [-1, -3]]

In [8]:
a.dominates(a[0])

True

In [9]:
a.dominates([-1.2, 1])

False

In [10]:
a._asserts()

In [11]:
b = NA(a)
print(b.merge([[-1.2, 1]]))
print(a.add_list([[-1.2, 1]]))
assert b == a
print(a.merge(b))

1
1
0


In [12]:
r = np.random.randn(100_000, 2).tolist()
r2 = sorted(np.random.randn(200, 2).tolist())
assert NA(r).add_list(r2) == NA(r).merge(r2)

In [13]:
%%timeit a = NA(r)
a.add_list(r2)

1000 loops, best of 3: 416 µs per loop


In [14]:
%%timeit a = NA(r)
a.merge(r2)

1000 loops, best of 3: 285 µs per loop


In [15]:
import numpy as np
def nondom_arch(n):
    return np.abs(np.linspace(-1, 1, 2 * n).reshape(2, n).T).tolist()
# nondom_arch(3)

## Timing of initialization

In [16]:
%timeit nondom_arch(1_000)
%timeit nondom_arch(10_000)
%timeit nondom_arch(100_000)  # just checking baseline

The slowest run took 4.15 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 96.3 µs per loop
1000 loops, best of 3: 843 µs per loop
100 loops, best of 3: 16.6 ms per loop


In [17]:
%timeit NA(nondom_arch(1_000))
%timeit NA(nondom_arch(10_000))
%timeit NA(nondom_arch(100_000))  # nondom_arch itself takes about 25%

1000 loops, best of 3: 1.03 ms per loop
100 loops, best of 3: 11.7 ms per loop
10 loops, best of 3: 117 ms per loop


In [18]:
randars = {}  # prepare input lists
for n in [1_000, 10_000, 100_000]:
    randars[n] = np.random.rand(n, 2).tolist()
len(NA(nondom_arch(100_000))), len(NA(randars[100_000]))

(100000, 9)

In [19]:
%timeit NA(randars[1_000])
%timeit NA(randars[10_000])
%timeit NA(randars[100_000])

1000 loops, best of 3: 751 µs per loop
100 loops, best of 3: 8.86 ms per loop
10 loops, best of 3: 167 ms per loop


In [20]:
%timeit sorted(nondom_arch(1_000))
%timeit sorted(nondom_arch(10_000))
%timeit sorted(nondom_arch(100_000))

10000 loops, best of 3: 132 µs per loop
1000 loops, best of 3: 1.38 ms per loop
10 loops, best of 3: 19.6 ms per loop


In [21]:
%timeit sorted(randars[1_000])
%timeit sorted(randars[10_000])
%timeit sorted(randars[100_000])

1000 loops, best of 3: 309 µs per loop
100 loops, best of 3: 4.67 ms per loop
10 loops, best of 3: 115 ms per loop


In [22]:
%timeit list(randars[1_000])
%timeit list(randars[10_000])
%timeit list(randars[100_000])

The slowest run took 5.21 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 2.32 µs per loop
10000 loops, best of 3: 35.1 µs per loop
1000 loops, best of 3: 773 µs per loop


### Summary with 1e5 data
```
   1 ms `list` 
  22 ms `sorted` on sorted list
 130 ms `sorted` on unsorted list
 110 ms archive on sorted nondominated list
 190 ms archive on list which needs pruning (was 1300ms)
```

## Timing of `add`

In [23]:
%%timeit a = NA(nondom_arch(1_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 7.11 ms per loop


In [24]:
%%timeit a = NA(nondom_arch(10_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 7.31 ms per loop


In [25]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in and makes it 20 times slower if implemented with pop

10 loops, best of 3: 20.4 ms per loop


In [26]:
%%timeit a = NA(nondom_arch(100_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a) # no deletion has taken place

100 loops, best of 3: 11.4 ms per loop


In [27]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-4 for ai in a[np.random.randint(len(a))]])
len(a)  # deletion kicks in

1 loop, best of 3: 253 ms per loop


In [28]:
%%timeit a = NA(nondom_arch(1_000_000))
for i in range(1000):
    a.add([ai - 1e-8 for ai in a[np.random.randint(len(a))]])
len(a)

100 loops, best of 3: 13.4 ms per loop


## Timing of Hypervolume computation

In [29]:
%timeit a = NA(nondom_arch(1_000), [5, 5])  # takes 4x longer than without hypervolume computation

100 loops, best of 3: 3.33 ms per loop


In [30]:
%timeit a = NA(nondom_arch(10_000), [5, 5])

10 loops, best of 3: 33.3 ms per loop


In [31]:
%timeit a = NA(nondom_arch(100_000), [5, 5])  # takes 3x longer than without hypervolume computation

1 loop, best of 3: 331 ms per loop


In [32]:
%%timeit a = NA(nondom_arch(1_000), [5, 5])
a.hypervolume

The slowest run took 14.22 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 219 ns per loop


In [33]:
%%timeit a = NA(nondom_arch(10_000), [5, 5])
a.hypervolume

The slowest run took 19.76 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 221 ns per loop


In [34]:
%%timeit a = NA(nondom_arch(100_000), [5, 5]) 
a.hypervolume

The slowest run took 18.42 times longer than the fastest. This could mean that an intermediate result is being cached.
1000000 loops, best of 3: 220 ns per loop
